In [64]:
import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize
import plotly.express as px
import plotly.graph_objects as go
import datetime

In [65]:
import warnings
warnings.filterwarnings('ignore')

In [66]:
approval = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vS-FKWVTTFtJT6u56e0bqdfoMcXvDO1DUChsJ3jQAMB2lZk2SMqVfmg7dGjclTYkYWz-Pm5lfcLPjp4/pub?output=csv')
# approval = approval.reset_index()
# approval = approval.rename({'index':'poll_id'}, axis=1)
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,tracking,approve,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp
0,Donald Trump (2nd term),Voters,InsiderAdvantage,NaN,5/17/2025,5/19/2025,1000.0,LV,1.161777,1.161777,NaN,54.6,44.4,10.2,53.07928,45.47200,7.607281,https://insideradvantage.com/trump-job-perform...,5/19/2025,21:15:54 20 May 2025
1,Donald Trump (2nd term),All polls,InsiderAdvantage,NaN,5/17/2025,5/19/2025,1000.0,LV,1.161777,1.161777,NaN,54.6,44.4,10.2,52.57597,45.96692,6.609051,https://insideradvantage.com/trump-job-perform...,5/19/2025,21:15:51 20 May 2025
2,Donald Trump (2nd term),Cost,Harris Insights & Analytics,Harvard CAPS,5/14/2025,5/15/2025,1903.0,RV,1.305367,1.082428,NaN,43.0,50.0,-7.0,41.19954,52.34557,-11.146030,https://harvardharrispoll.com/key-results-may-5/,5/15/2025,21:16:02 20 May 2025
3,Donald Trump (2nd term),Strong,Harris Insights & Analytics,Harvard CAPS,5/14/2025,5/15/2025,1903.0,RV,1.305367,1.082428,NaN,30.0,36.0,-6.0,29.98970,38.91243,-8.922729,https://harvardharrispoll.com/key-results-may-5/,5/15/2025,21:15:57 20 May 2025
4,Donald Trump (2nd term),Economy,Harris Insights & Analytics,Harvard CAPS,5/14/2025,5/15/2025,1903.0,RV,1.305367,1.082428,NaN,46.0,48.0,-2.0,45.01950,49.59774,-4.578239,https://harvardharrispoll.com/key-results-may-5/,5/15/2025,21:16:00 20 May 2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1277,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,2/11/2025,2/17/2025,1500.0,LV,0.105676,0.000000,True,52.0,46.0,6.0,49.93290,46.85780,3.075100,https://www.rasmussenreports.com/public_conten...,2/17/2025,21:15:54 20 May 2025
1278,Donald Trump (2nd term),Economy,YouGov,Economist,2/23/2025,2/25/2025,1604.0,A,0.546871,0.000000,NaN,45.0,43.0,2.0,44.89407,44.96550,-0.071430,https://d3nkl3psvxxpe9.cloudfront.net/document...,2/25/2025,21:16:00 20 May 2025
1279,Donald Trump (2nd term),Voters,Targoz Market Research,Overton Insights,3/24/2025,3/28/2025,1200.0,RV,0.951969,0.000000,NaN,46.0,51.0,-5.0,46.33902,50.60021,-4.261189,https://overtoninsights.com/wp-content/uploads...,3/28/2025,21:15:54 20 May 2025
1280,Donald Trump (2nd term),Voters,Rasmussen Reports,NaN,3/7/2025,3/13/2025,1500.0,LV,0.316196,0.000000,True,52.0,47.0,5.0,49.93290,47.85780,2.075100,https://www.rasmussenreports.com/public_conten...,3/13/2025,21:15:54 20 May 2025


In [67]:
approval = approval[~approval['subgroup'].isin(['Weak', 'Strong', 'Economy', 'Cost', 'Immigration', 'Trade', 'Adults', 'Voters'])]

In [68]:
approval.to_csv('trump_approval_polls.csv')

In [69]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'enddate', 'samplesize', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp'], dtype=object)

In [70]:
approval.shape

(282, 20)

In [71]:
# dates = approval['Dates'].str.extract(r'(\d{1,2}\/\d{1,2}) \- (\d{1,2}\/\d{1,2}), (\d{4})')
# dates

In [72]:
# dates['end_date'] = dates[1] + '/' + dates[2]
# dates['end_date'] = pd.to_datetime(dates['end_date'])
# dates

In [73]:
# approval['Dates'] = dates['end_date']
# approval.head()

In [74]:
# samples = approval['Sample'].str.extract(r'[0-9,]{3,5} (\w{1,2})@@(\d{3,4})')
# samples

In [75]:
# approval['population'] = samples[0]
# approval['sample_size'] = samples[1].astype(int)
# approval.head()

In [76]:
ratings = pd.read_csv('pollster_ratings_silver.csv')
ratings

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
0,AtlasIntel,A+@@1,-0.957740,D+0.16@@0.16334188,91,D+
1,Marquette University Law School,A/B@@4,-0.920261,D+0.32@@0.31545374,17,D+
2,The Washington Post,A+@@1,-0.902281,D+0.90@@0.89811021,98,D+
3,Research & Polling Inc.,A@@2,-0.871136,R+0.55@@-0.54944503,44,R+
4,The New York Times/Siena College,A@@2,-0.763947,D+1.41@@1.4121077,116,D+
...,...,...,...,...,...,...
530,Nielson Brothers Polling,C/D@@12,1.946864,D+3.05@@3.0515239,11,D+
531,Ascend Action,C/D@@12,2.282717,D+3.35@@3.3485439,6,D+
532,Jayhawk Consulting Services,C/D@@12,2.353968,D+2.27@@2.2674108,2,D+
533,Fort Hays State University,C/D@@12,2.489170,R+1.10@@-1.0960908,6,R+


In [77]:
ratings['Pollster'].values

array(['AtlasIntel', 'Marquette University Law School',
       'The Washington Post', 'Research & Polling Inc.',
       'The New York Times/Siena College', 'Cygnal', 'SurveyUSA',
       'Siena College', 'Beacon Research/Shaw & Co. Research',
       'Alaska Survey Research', 'OnMessage Inc.', 'TIPP Insights',
       'Emerson College', 'Landmark Communications',
       'Monmouth University', 'Data Orbital',
       'University of North Florida', 'MassINC Polling Group',
       'Marist College', 'Public Policy Institute of California',
       'Public Policy Polling', 'University of California Berkeley',
       'CBS News/The New York Times', 'Christopher Newport University',
       'Mason-Dixon Polling & Strategy', 'Strategic Vision LLC',
       'Suffolk University', 'Stockton University',
       'Field Research Corp. (Field Poll)', 'Quinnipiac University',
       'Los Angeles Times',
       'Hart Research Associates/Public Opinion Strategies',
       'Public Opinion Strategies', 'Susquehan

In [78]:
ratings[ratings['Pollster'].str.contains('North Star')]#.sum()

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
75,North Star Opinion Research,B/C@@8,0.043339,R+0.63@@-0.62697512,13,R+
425,Univision/Latino Decisions/North Star Opinion ...,B/C@@8,0.830409,D+1.06@@1.0564657,7,D+


In [79]:
banned_pollsters = ratings[ratings['Grade'].str.contains('F')]['Pollster'].values
banned_pollsters

array(['Strategic Vision LLC', 'Pharos Research Group', 'Research 2000',
       'Big Data Poll', 'Overtime Politics', 'Rethink Priorities',
       'Blumenthal Research Daily', 'CSP Polling', 'KG Polling',
       'OurProgress (The Progress Campaign)', 'TCJ Research'],
      dtype=object)

In [80]:
approval = approval[~approval['pollster'].isin(banned_pollsters)]
approval.shape

(282, 20)

In [81]:
approval = pd.merge(left=approval, right=ratings[['Pollster', 'Predictive Plus-Minus']], left_on='pollster',
                   right_on='Pollster', how='inner')
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),All polls,InsiderAdvantage,NaN,5/17/2025,5/19/2025,1000.0,LV,1.161777,1.161777,...,44.4,10.2,52.57597,45.96692,6.609051,https://insideradvantage.com/trump-job-perform...,5/19/2025,21:15:51 20 May 2025,InsiderAdvantage,0.034562
1,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,5/14/2025,5/15/2025,1903.0,RV,1.305367,1.082428,...,48.0,-1.0,45.83914,49.80199,-3.962849,https://harvardharrispoll.com/key-results-may-5/,5/15/2025,21:15:51 20 May 2025,Harris Insights & Analytics,0.162447
2,Donald Trump (2nd term),All polls,YouGov,Economist,5/16/2025,5/19/2025,1710.0,A,0.858229,0.858229,...,51.0,-8.0,43.67791,51.60490,-7.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/19/2025,21:15:51 20 May 2025,YouGov,0.047635
3,Donald Trump (2nd term),All polls,J.L. Partners,Daily Mail,5/13/2025,5/14/2025,1003.0,RV,1.035558,0.794274,...,46.0,-1.0,45.65443,49.07562,-3.421188,https://x.com/J_L_Partners/status/192377859529...,5/14/2025,21:15:51 20 May 2025,J.L. Partners,0.197911
4,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,5/13/2025,5/19/2025,1500.0,LV,0.803705,0.704391,...,50.0,-1.0,46.57432,51.10401,-4.529690,https://www.rasmussenreports.com/public_conten...,5/19/2025,21:15:51 20 May 2025,Rasmussen Reports,0.087015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,Donald Trump (2nd term),All polls,YouGov,CBS News,4/8/2025,4/11/2025,2410.0,A,0.506873,0.000000,...,53.0,-6.0,47.67791,53.60490,-5.926991,https://www.cbsnews.com/news/poll-trump-tariff...,4/11/2025,21:15:51 20 May 2025,YouGov,0.047635
265,Donald Trump (2nd term),All polls,Fabrizio Lee & Associates/Impact Research,Wall Street Journal,3/27/2025,4/1/2025,1500.0,RV,1.054110,0.000000,...,51.0,-5.0,46.20646,50.62058,-4.414120,https://prod-i.a.dj.com/public/resources/docum...,4/1/2025,21:15:51 20 May 2025,Fabrizio Lee & Associates/Impact Research,0.588291
266,Donald Trump (2nd term),All polls,Public Opinion Strategies,Paragon Health Institute,2/25/2025,3/2/2025,1000.0,RV,1.268962,0.000000,...,49.0,2.0,50.06641,48.81849,1.247921,https://paragoninstitute.org/medicaid/paragon-...,3/2/2025,21:15:51 20 May 2025,Public Opinion Strategies,-0.246974
267,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,3/6/2025,3/12/2025,1500.0,LV,0.105631,0.000000,...,47.0,5.0,49.57432,48.10401,1.470310,https://www.rasmussenreports.com/public_conten...,3/12/2025,21:15:51 20 May 2025,Rasmussen Reports,0.087015


In [82]:
approval.shape

(269, 22)

In [83]:
approval['Pollster'].value_counts()

Pollster
Rasmussen Reports                                     83
YouGov                                                33
Morning Consult                                       21
RMG Research                                          17
Ipsos                                                 14
J.L. Partners                                          8
TIPP Insights                                          6
Echelon Insights                                       6
Harris Insights & Analytics                            5
Emerson College                                        5
Global Strategy Group/GBAO (Navigator Research)        5
SSRS                                                   4
Quinnipiac University                                  4
AtlasIntel                                             4
Gallup                                                 4
McLaughlin & Associates                                4
Cygnal                                                 3
Clarity Campaign Labs 

In [84]:
# approval[~approval['tracking'].isna()]

In [85]:
approval['tracking'].value_counts()

tracking
True    86
Name: count, dtype: int64

In [86]:
approval = approval[approval['tracking'].isna() & (~approval['samplesize'].isna())]
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),All polls,InsiderAdvantage,NaN,5/17/2025,5/19/2025,1000.0,LV,1.161777,1.161777,...,44.4,10.2,52.57597,45.96692,6.609051,https://insideradvantage.com/trump-job-perform...,5/19/2025,21:15:51 20 May 2025,InsiderAdvantage,0.034562
1,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,5/14/2025,5/15/2025,1903.0,RV,1.305367,1.082428,...,48.0,-1.0,45.83914,49.80199,-3.962849,https://harvardharrispoll.com/key-results-may-5/,5/15/2025,21:15:51 20 May 2025,Harris Insights & Analytics,0.162447
2,Donald Trump (2nd term),All polls,YouGov,Economist,5/16/2025,5/19/2025,1710.0,A,0.858229,0.858229,...,51.0,-8.0,43.67791,51.60490,-7.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/19/2025,21:15:51 20 May 2025,YouGov,0.047635
3,Donald Trump (2nd term),All polls,J.L. Partners,Daily Mail,5/13/2025,5/14/2025,1003.0,RV,1.035558,0.794274,...,46.0,-1.0,45.65443,49.07562,-3.421188,https://x.com/J_L_Partners/status/192377859529...,5/14/2025,21:15:51 20 May 2025,J.L. Partners,0.197911
5,Donald Trump (2nd term),All polls,Ipsos,Reuters,5/16/2025,5/18/2025,1024.0,A,0.722075,0.679312,...,52.0,-10.0,44.02072,50.98790,-6.967178,https://www.reuters.com/world/us/trumps-approv...,5/18/2025,21:15:51 20 May 2025,Ipsos,0.442898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Donald Trump (2nd term),All polls,YouGov,CBS News,2/5/2025,2/7/2025,2175.0,A,0.588896,0.000000,...,47.0,6.0,53.67791,47.60490,6.073009,https://www.cbsnews.com/news/trump-approval-op...,2/7/2025,21:15:51 20 May 2025,YouGov,0.047635
261,Donald Trump (2nd term),All polls,Quinnipiac University,NaN,2/13/2025,2/17/2025,1039.0,RV,1.236978,0.000000,...,49.0,-4.0,46.80139,48.40109,-1.599697,https://poll.qu.edu/poll-release?releaseid=3919,2/17/2025,21:15:51 20 May 2025,Quinnipiac University,-0.290484
264,Donald Trump (2nd term),All polls,YouGov,CBS News,4/8/2025,4/11/2025,2410.0,A,0.506873,0.000000,...,53.0,-6.0,47.67791,53.60490,-5.926991,https://www.cbsnews.com/news/poll-trump-tariff...,4/11/2025,21:15:51 20 May 2025,YouGov,0.047635
265,Donald Trump (2nd term),All polls,Fabrizio Lee & Associates/Impact Research,Wall Street Journal,3/27/2025,4/1/2025,1500.0,RV,1.054110,0.000000,...,51.0,-5.0,46.20646,50.62058,-4.414120,https://prod-i.a.dj.com/public/resources/docum...,4/1/2025,21:15:51 20 May 2025,Fabrizio Lee & Associates/Impact Research,0.588291


In [87]:
auxiliary = pd.read_csv('auxiliary_pollster_rating_stats.csv')
auxiliary

,Pollster,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat
0,AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+
1,The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+
2,Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+
3,The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+
4,Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+
...,...,...,...,...,...,...,...,...
93,Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+
94,Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+
95,Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+
96,McLaughlin & Associates,0.689655,34.5%@@65.517241,R+4.94@@-4.9392128,6.2@@6.17804,0.000000,1,R+


In [88]:
house_eff = auxiliary['House effect'].str.extract(r'(\w)\+(\d\.\d\d)@@*.')
house_eff[1] = house_eff[1].astype(float)
house_eff

,0,1
0,R,2.00
1,D,1.02
2,D,0.63
3,D,0.79
4,R,0.60
...,...,...
93,R,2.44
94,R,0.45
95,R,3.23
96,R,4.94


In [89]:
# R --> -1, D --> 1
house_eff[0] = house_eff[0].replace({'R':-1, 'D':1})
house_eff['house_effect'] = house_eff[0] * house_eff[1]
house_eff

,0,1,house_effect
0,-1,2.00,-2.00
1,1,1.02,1.02
2,1,0.63,0.63
3,1,0.79,0.79
4,-1,0.60,-0.60
...,...,...,...
93,-1,2.44,-2.44
94,-1,0.45,-0.45
95,-1,3.23,-3.23
96,-1,4.94,-4.94


In [90]:
auxiliary['house_effect'] = house_eff['house_effect']
auxiliary = auxiliary.set_index(['Pollster'])
auxiliary

,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat,house_effect
Pollster,,,,,,,,
AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+,-2.00
The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+,1.02
Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+,0.63
The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+,0.79
Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+,-0.60
...,...,...,...,...,...,...,...,...
Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+,-2.44
Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+,-0.45
Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+,-3.23


In [91]:
def apply_house_eff(data, pollster):
    df = data[data['pollster']==pollster]
    try:
        df['approve_adj'] = df['approve'] + (auxiliary.loc[pollster, 'house_effect'] / 2)
        df['disapprove_adj'] = df['disapprove'] - (auxiliary.loc[pollster, 'house_effect'] / 2)
    except KeyError:
        df['approve_adj'] = df['approve']
        df['disapprove_adj'] = df['disapprove']
    return df

In [92]:
pollsters = approval['pollster'].value_counts().index.values
df_adjs = []
for p in pollsters:
    df = apply_house_eff(approval, p)
    df_adjs.append(df)

approval_adj = pd.concat(df_adjs, axis=0)
approval_adj

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj
2,Donald Trump (2nd term),All polls,YouGov,Economist,5/16/2025,5/19/2025,1710.0,A,0.858229,0.858229,...,43.67791,51.60490,-7.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/19/2025,21:15:51 20 May 2025,YouGov,0.047635,43.250,50.750
11,Donald Trump (2nd term),All polls,YouGov,Economist,5/9/2025,5/12/2025,1786.0,A,0.638945,0.350111,...,43.67791,52.60490,-8.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/12/2025,21:15:51 20 May 2025,YouGov,0.047635,43.250,51.750
18,Donald Trump (2nd term),All polls,YouGov,NaN,5/6/2025,5/8/2025,1143.0,A,0.468114,0.132433,...,42.67791,50.60490,-7.926991,https://ygo-assets-websites-editorial-emea.you...,5/8/2025,21:15:51 20 May 2025,YouGov,0.047635,42.250,49.750
20,Donald Trump (2nd term),All polls,YouGov,Economist,5/2/2025,5/5/2025,1850.0,A,0.503410,0.114052,...,42.67791,52.60490,-9.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/5/2025,21:15:51 20 May 2025,YouGov,0.047635,42.250,51.750
32,Donald Trump (2nd term),All polls,YouGov,NaN,4/25/2025,5/2/2025,2220.0,A,0.457506,0.057106,...,42.67791,52.60490,-9.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/2/2025,21:15:51 20 May 2025,YouGov,0.047635,42.250,51.750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,Donald Trump (2nd term),All polls,Targoz Market Research,Overton Insights,3/24/2025,3/28/2025,1200.0,RV,0.951969,0.000000,...,46.16313,50.68949,-4.526360,https://overtoninsights.com/wp-content/uploads...,3/28/2025,21:15:51 20 May 2025,Targoz Market Research,0.731655,46.570,50.430
140,Donald Trump (2nd term),All polls,GBAO,Third Way,3/17/2025,3/23/2025,2000.0,RV,1.210367,0.000000,...,47.63491,50.09224,-2.457329,https://www.thirdway.org/memo/breaking-through...,3/23/2025,21:15:51 20 May 2025,GBAO,0.384788,48.000,50.000
122,Donald Trump (2nd term),All polls,SurveyMonkey,NBC News,4/11/2025,4/20/2025,19682.0,A,1.236225,0.000000,...,44.80983,54.04432,-9.234489,https://www.nbcnews.com/politics/trump-adminis...,4/20/2025,21:15:51 20 May 2025,SurveyMonkey,1.047575,45.915,54.085
115,Donald Trump (2nd term),All polls,University of Massachusetts (Amherst),WCVB-TV,4/4/2025,4/9/2025,1000.0,A,0.983206,0.000000,...,44.28984,51.06049,-6.770649,https://www.umass.edu/political-science/about/...,4/9/2025,21:15:51 20 May 2025,University of Massachusetts (Amherst),0.536200,44.000,51.000


In [93]:
approval.shape

(183, 22)

In [94]:
approval = approval_adj.copy()
approval.shape

(183, 24)

In [95]:
approval['poll_id'] = pd.factorize(approval['pollster'] + approval['enddate'])[0]
approval.head(7)

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj,poll_id
2,Donald Trump (2nd term),All polls,YouGov,Economist,5/16/2025,5/19/2025,1710.0,A,0.858229,0.858229,...,51.6049,-7.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/19/2025,21:15:51 20 May 2025,YouGov,0.047635,43.25,50.75,0
11,Donald Trump (2nd term),All polls,YouGov,Economist,5/9/2025,5/12/2025,1786.0,A,0.638945,0.350111,...,52.6049,-8.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/12/2025,21:15:51 20 May 2025,YouGov,0.047635,43.25,51.75,1
18,Donald Trump (2nd term),All polls,YouGov,NaN,5/6/2025,5/8/2025,1143.0,A,0.468114,0.132433,...,50.6049,-7.926991,https://ygo-assets-websites-editorial-emea.you...,5/8/2025,21:15:51 20 May 2025,YouGov,0.047635,42.25,49.75,2
20,Donald Trump (2nd term),All polls,YouGov,Economist,5/2/2025,5/5/2025,1850.0,A,0.503410,0.114052,...,52.6049,-9.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/5/2025,21:15:51 20 May 2025,YouGov,0.047635,42.25,51.75,3
32,Donald Trump (2nd term),All polls,YouGov,NaN,4/25/2025,5/2/2025,2220.0,A,0.457506,0.057106,...,52.6049,-9.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/2/2025,21:15:51 20 May 2025,YouGov,0.047635,42.25,51.75,4
39,Donald Trump (2nd term),All polls,YouGov,Economist,4/25/2025,4/28/2025,1785.0,A,0.400127,0.032139,...,52.6049,-9.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,4/28/2025,21:15:51 20 May 2025,YouGov,0.047635,42.25,51.75,5
40,Donald Trump (2nd term),All polls,YouGov,Yahoo News,4/25/2025,4/28/2025,1597.0,A,0.382705,0.030739,...,53.6049,-10.926990,https://www.scribd.com/document/855435184/2025...,4/28/2025,21:15:51 20 May 2025,YouGov,0.047635,42.25,52.75,5


In [96]:
approval['population'] = approval['population'].str.strip(' ')

In [97]:
def pipeline(data: pd.DataFrame):
    dup = data[data.duplicated(subset='poll_id', keep=False)]
    ndup = data[~data.duplicated(subset='poll_id', keep=False)]
    # print('dup', dup, '\n ndup', ndup)
    dup_a = dup[dup['population'].isin(['A'])]
    # print('dup_a', dup_a)
    df = pd.concat([dup_a, ndup], axis=0)
    
    dup = df[df.duplicated(subset='poll_id', keep=False)]
    ndup = df[~df.duplicated(subset='poll_id', keep=False)]
    dup_lv = dup[dup['population'].isin(['RV'])]
    # print('dup_rv', dup_lv, 'ndup', ndup)
    df = pd.concat([dup_lv, ndup], axis=0)
    
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    return df

In [98]:
approval = approval.rename({'enddate':'end_date', 'samplesize':'sample_size', 'Predictive Plus-Minus':'pred_plus_minus'}, axis=1)

In [99]:
approval['population'].value_counts().index.values

array(['RV', 'A', 'LV', 'V'], dtype=object)

In [100]:
approval.shape, pipeline(approval).shape

((183, 25), (175, 25))

In [101]:
pipeline(approval)

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
2,Donald Trump (2nd term),All polls,YouGov,Economist,5/16/2025,2025-05-19,1710.0,A,0.858229,0.858229,...,51.60490,-7.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/19/2025,21:15:51 20 May 2025,YouGov,0.047635,43.250,50.750,0
11,Donald Trump (2nd term),All polls,YouGov,Economist,5/9/2025,2025-05-12,1786.0,A,0.638945,0.350111,...,52.60490,-8.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/12/2025,21:15:51 20 May 2025,YouGov,0.047635,43.250,51.750,1
18,Donald Trump (2nd term),All polls,YouGov,NaN,5/6/2025,2025-05-08,1143.0,A,0.468114,0.132433,...,50.60490,-7.926991,https://ygo-assets-websites-editorial-emea.you...,5/8/2025,21:15:51 20 May 2025,YouGov,0.047635,42.250,49.750,2
20,Donald Trump (2nd term),All polls,YouGov,Economist,5/2/2025,2025-05-05,1850.0,A,0.503410,0.114052,...,52.60490,-9.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/5/2025,21:15:51 20 May 2025,YouGov,0.047635,42.250,51.750,3
32,Donald Trump (2nd term),All polls,YouGov,NaN,4/25/2025,2025-05-02,2220.0,A,0.457506,0.057106,...,52.60490,-9.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/2/2025,21:15:51 20 May 2025,YouGov,0.047635,42.250,51.750,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,Donald Trump (2nd term),All polls,Targoz Market Research,Overton Insights,3/24/2025,2025-03-28,1200.0,RV,0.951969,0.000000,...,50.68949,-4.526360,https://overtoninsights.com/wp-content/uploads...,3/28/2025,21:15:51 20 May 2025,Targoz Market Research,0.731655,46.570,50.430,174
140,Donald Trump (2nd term),All polls,GBAO,Third Way,3/17/2025,2025-03-23,2000.0,RV,1.210367,0.000000,...,50.09224,-2.457329,https://www.thirdway.org/memo/breaking-through...,3/23/2025,21:15:51 20 May 2025,GBAO,0.384788,48.000,50.000,175
122,Donald Trump (2nd term),All polls,SurveyMonkey,NBC News,4/11/2025,2025-04-20,19682.0,A,1.236225,0.000000,...,54.04432,-9.234489,https://www.nbcnews.com/politics/trump-adminis...,4/20/2025,21:15:51 20 May 2025,SurveyMonkey,1.047575,45.915,54.085,176
115,Donald Trump (2nd term),All polls,University of Massachusetts (Amherst),WCVB-TV,4/4/2025,2025-04-09,1000.0,A,0.983206,0.000000,...,51.06049,-6.770649,https://www.umass.edu/political-science/about/...,4/9/2025,21:15:51 20 May 2025,University of Massachusetts (Amherst),0.536200,44.000,51.000,177


In [102]:
approval[approval['pollster']=='Harris Insights & Analytics']

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
1,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,5/14/2025,5/15/2025,1903.0,RV,1.305367,1.082428,...,49.80199,-3.962849,https://harvardharrispoll.com/key-results-may-5/,5/15/2025,21:15:51 20 May 2025,Harris Insights & Analytics,0.162447,47.105,47.895,128
76,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,4/9/2025,4/10/2025,2286.0,RV,0.950531,0.000000,...,47.80199,-0.962849,https://harvardharrispoll.com/wp-content/uploa...,4/10/2025,21:15:51 20 May 2025,Harris Insights & Analytics,0.162447,48.105,45.895,129
238,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,2/19/2025,2/20/2025,2443.0,RV,1.382092,0.000000,...,44.80199,6.037151,https://harvardharrispoll.com/wp-content/uploa...,2/20/2025,21:15:51 20 May 2025,Harris Insights & Analytics,0.162447,52.105,42.895,130
257,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,3/26/2025,3/27/2025,2746.0,RV,1.124949,0.000000,...,47.80199,0.037151,https://harvardharrispoll.com/key-results-marc...,3/27/2025,21:15:51 20 May 2025,Harris Insights & Analytics,0.162447,49.105,45.895,131


In [103]:
# banned_pollsters_aux = ['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs', 'Trafalgar Group', 'OnMessage Inc.']
banned_pollsters_aux = pd.read_csv('banned_pollsters_aux.csv')['banned_pollsters'].values
banned_pollsters_aux

array(['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs',
       'Trafalgar Group', 'OnMessage Inc.', 'McLaughlin & Associates',
       'North Star Opinion Research', 'Trafalgar Group/Insider Advantage'],
      dtype=object)

In [104]:
approval = approval[~approval['pollster'].isin(banned_pollsters_aux)]
approval.head()

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
2,Donald Trump (2nd term),All polls,YouGov,Economist,5/16/2025,5/19/2025,1710.0,A,0.858229,0.858229,...,51.6049,-7.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/19/2025,21:15:51 20 May 2025,YouGov,0.047635,43.25,50.75,0
11,Donald Trump (2nd term),All polls,YouGov,Economist,5/9/2025,5/12/2025,1786.0,A,0.638945,0.350111,...,52.6049,-8.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/12/2025,21:15:51 20 May 2025,YouGov,0.047635,43.25,51.75,1
18,Donald Trump (2nd term),All polls,YouGov,NaN,5/6/2025,5/8/2025,1143.0,A,0.468114,0.132433,...,50.6049,-7.926991,https://ygo-assets-websites-editorial-emea.you...,5/8/2025,21:15:51 20 May 2025,YouGov,0.047635,42.25,49.75,2
20,Donald Trump (2nd term),All polls,YouGov,Economist,5/2/2025,5/5/2025,1850.0,A,0.503410,0.114052,...,52.6049,-9.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/5/2025,21:15:51 20 May 2025,YouGov,0.047635,42.25,51.75,3
32,Donald Trump (2nd term),All polls,YouGov,NaN,4/25/2025,5/2/2025,2220.0,A,0.457506,0.057106,...,52.6049,-9.926991,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/2/2025,21:15:51 20 May 2025,YouGov,0.047635,42.25,51.75,4


In [105]:
approval[approval['pollster']=='Harris Insights & Analytics']

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
1,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,5/14/2025,5/15/2025,1903.0,RV,1.305367,1.082428,...,49.80199,-3.962849,https://harvardharrispoll.com/key-results-may-5/,5/15/2025,21:15:51 20 May 2025,Harris Insights & Analytics,0.162447,47.105,47.895,128
76,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,4/9/2025,4/10/2025,2286.0,RV,0.950531,0.000000,...,47.80199,-0.962849,https://harvardharrispoll.com/wp-content/uploa...,4/10/2025,21:15:51 20 May 2025,Harris Insights & Analytics,0.162447,48.105,45.895,129
238,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,2/19/2025,2/20/2025,2443.0,RV,1.382092,0.000000,...,44.80199,6.037151,https://harvardharrispoll.com/wp-content/uploa...,2/20/2025,21:15:51 20 May 2025,Harris Insights & Analytics,0.162447,52.105,42.895,130
257,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,3/26/2025,3/27/2025,2746.0,RV,1.124949,0.000000,...,47.80199,0.037151,https://harvardharrispoll.com/key-results-marc...,3/27/2025,21:15:51 20 May 2025,Harris Insights & Analytics,0.162447,49.105,45.895,131


In [106]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'end_date', 'sample_size', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp', 'Pollster', 'pred_plus_minus', 'approve_adj',
       'disapprove_adj', 'poll_id'], dtype=object)

In [107]:
approval_polls = approval[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'approve_adj', 'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id']]
approval_polls

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id
2,YouGov,Economist,5/16/2025,5/19/2025,1710.0,A,43.0,51.0,43.250,50.750,-8.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,0
11,YouGov,Economist,5/9/2025,5/12/2025,1786.0,A,43.0,52.0,43.250,51.750,-9.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,1
18,YouGov,NaN,5/6/2025,5/8/2025,1143.0,A,42.0,50.0,42.250,49.750,-8.0,https://ygo-assets-websites-editorial-emea.you...,0.047635,2
20,YouGov,Economist,5/2/2025,5/5/2025,1850.0,A,42.0,52.0,42.250,51.750,-10.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,3
32,YouGov,NaN,4/25/2025,5/2/2025,2220.0,A,42.0,52.0,42.250,51.750,-10.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,Targoz Market Research,Overton Insights,3/24/2025,3/28/2025,1200.0,RV,46.0,51.0,46.570,50.430,-5.0,https://overtoninsights.com/wp-content/uploads...,0.731655,174
140,GBAO,Third Way,3/17/2025,3/23/2025,2000.0,RV,48.0,50.0,48.000,50.000,-2.0,https://www.thirdway.org/memo/breaking-through...,0.384788,175
122,SurveyMonkey,NBC News,4/11/2025,4/20/2025,19682.0,A,45.0,55.0,45.915,54.085,-10.0,https://www.nbcnews.com/politics/trump-adminis...,1.047575,176
115,University of Massachusetts (Amherst),WCVB-TV,4/4/2025,4/9/2025,1000.0,A,44.0,51.0,44.000,51.000,-7.0,https://www.umass.edu/political-science/about/...,0.536200,177


In [108]:
def avgs_pipeline(data: pd.DataFrame, date: datetime.date):
    state_pivot = pipeline(data)
    # state_pivot['end_date'] = pd.to_datetime(state_pivot['end_date'])
    # state_pivot['pred_plus_minus'] = state_pivot['pred_plus_minus'].map(lambda x: -x)
    state_pivot = state_pivot[state_pivot['end_date'] <= pd.to_datetime(date)]
    state_pivot['population'] = state_pivot['population'].astype(str).map(lambda x: x.strip())
    
    total_num_polls = state_pivot.shape[0]

    # Sample size weights
    total_sample_size = np.sum(state_pivot['sample_size'])
    state_pivot['sample_size_weights'] = (state_pivot['sample_size'].map(lambda x: np.sqrt(min(x, 3000))) / np.sqrt(np.median(state_pivot['sample_size'].map(lambda x: min(x, 2000)))))
    # print(np.median(state_pivot['sample_size'].map(lambda x: min(x, 5000))))
    state_pivot['sample_size_weights'] /= np.sum(state_pivot['sample_size_weights'])
    
    # Time weights
    # Variation of the equation used here: https://polls.votehub.us/
    latest_date = pd.Timestamp(date)
    delta = state_pivot['end_date'].apply(lambda x: (latest_date - x).days)# (latest_date - state_pivot['end_date']).apply(lambda x: x.days)
    linear_weights = (1 - delta/((latest_date - state_pivot['end_date'].min()).days + 1))
    exp_weights = 0.1**(delta/(40))
    state_pivot['time_weights'] =  0 * linear_weights + 1 * exp_weights
    # state_pivot['time_weights'] /= np.sum(state_pivot['time_weights'])
    
    # Quality weights
    rel_quality = state_pivot['pred_plus_minus']
    def quality_weight(rel_qual):
        if rel_qual > 0.5: # Utilizing predictive plus-minus from Silver Bulletin's ratings
            return 0.2
        # return np.sqrt(-rel_qual+1)+0.05
        return np.exp(-rel_qual/1.3)
    state_pivot['quality_weights'] = rel_quality.map(quality_weight)
    # state_pivot['quality_weights'] /= np.sum(state_pivot['quality_weights'])
    
    # Population weights
    def population_weight(population):
        if population == 'RV':
            return 0.8
        elif population == 'LV':
            return 0.6
        return 1
    state_pivot['population_weights'] = state_pivot['population'].map(population_weight)
    # state_pivot['population_weights'] /= np.sum(state_pivot['population_weights'])

    # Gather the weights together
    state_pivot['total_weights'] = state_pivot['sample_size_weights'] * state_pivot['time_weights'] * state_pivot['quality_weights'] * state_pivot['population_weights']
    state_pivot['total_weights'] /= np.sum(state_pivot['total_weights']) # Normalization step
    
    return state_pivot

In [109]:
approval_polls['sample_size'].isna().any()

False

In [110]:
avgs_pipeline(approval_polls, datetime.datetime.today())

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id,sample_size_weights,time_weights,quality_weights,population_weights,total_weights
2,YouGov,Economist,5/16/2025,2025-05-19,1710.0,A,43.0,51.0,43.250,50.750,-8.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,0,0.006510,0.944061,0.964021,1.0,0.069605
11,YouGov,Economist,5/9/2025,2025-05-12,1786.0,A,43.0,52.0,43.250,51.750,-9.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,1,0.006653,0.630957,0.964021,1.0,0.047542
18,YouGov,NaN,5/6/2025,2025-05-08,1143.0,A,42.0,50.0,42.250,49.750,-8.0,https://ygo-assets-websites-editorial-emea.you...,0.047635,2,0.005322,0.501187,0.964021,1.0,0.030211
20,YouGov,Economist,5/2/2025,2025-05-05,1850.0,A,42.0,52.0,42.250,51.750,-10.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,3,0.006771,0.421697,0.964021,1.0,0.032339
32,YouGov,NaN,4/25/2025,2025-05-02,2220.0,A,42.0,52.0,42.250,51.750,-10.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,4,0.007418,0.354813,0.964021,1.0,0.029807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,Targoz Market Research,Overton Insights,3/24/2025,2025-03-28,1200.0,RV,46.0,51.0,46.570,50.430,-5.0,https://overtoninsights.com/wp-content/uploads...,0.731655,174,0.005453,0.047315,0.200000,0.8,0.000485
140,GBAO,Third Way,3/17/2025,2025-03-23,2000.0,RV,48.0,50.0,48.000,50.000,-2.0,https://www.thirdway.org/memo/breaking-through...,0.384788,175,0.007040,0.035481,0.743794,0.8,0.001746
122,SurveyMonkey,NBC News,4/11/2025,2025-04-20,19682.0,A,45.0,55.0,45.915,54.085,-10.0,https://www.nbcnews.com/politics/trump-adminis...,1.047575,176,0.008623,0.177828,0.200000,1.0,0.003603
115,University of Massachusetts (Amherst),WCVB-TV,4/4/2025,2025-04-09,1000.0,A,44.0,51.0,44.000,51.000,-7.0,https://www.umass.edu/political-science/about/...,0.536200,177,0.004978,0.094406,0.200000,1.0,0.001104


In [111]:
def approval_over_time(data: pd.DataFrame):
    date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
    yes_averages = []
    no_averages = []
    yes_stdevs = []
    no_stdevs = []
    for date in date_range:
        pipelined_df = avgs_pipeline(data, date)
        yes_avg = np.sum(pipelined_df['approve_adj'] * pipelined_df['total_weights'])
        no_avg = np.sum(pipelined_df['disapprove_adj'] * pipelined_df['total_weights'])
        yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve_adj'] - yes_avg)**2))
        no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove_adj'] - no_avg)**2))
        yes_averages.append(yes_avg)
        no_averages.append(no_avg)
        yes_stdevs.append(yes_std)
        no_stdevs.append(no_std)
    
    return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())

In [112]:
approval_avg, approval_std, polls_df = approval_over_time(approval_polls)
approval_avg

,Date,Approve,Disapprove
0,2025-01-23,51.310566,45.653535
1,2025-01-24,51.237797,45.330152
2,2025-01-25,51.237797,45.330152
3,2025-01-26,50.599984,45.294367
4,2025-01-27,49.443088,45.451175
...,...,...,...
113,2025-05-16,45.222849,50.610528
114,2025-05-17,45.222849,50.610528
115,2025-05-18,45.102544,50.653110
116,2025-05-19,45.308822,50.458753


In [113]:
polls_df.shape

(157, 19)

In [114]:
polls_df.head(3)

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id,sample_size_weights,time_weights,quality_weights,population_weights,total_weights
2,YouGov,Economist,5/16/2025,2025-05-19,1710.0,A,43.0,51.0,43.25,50.75,-8.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,0,0.006510,0.944061,0.964021,1.0,0.069605
11,YouGov,Economist,5/9/2025,2025-05-12,1786.0,A,43.0,52.0,43.25,51.75,-9.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,1,0.006653,0.630957,0.964021,1.0,0.047542
18,YouGov,NaN,5/6/2025,2025-05-08,1143.0,A,42.0,50.0,42.25,49.75,-8.0,https://ygo-assets-websites-editorial-emea.you...,0.047635,2,0.005322,0.501187,0.964021,1.0,0.030211


In [115]:
polls_df['net_adj'] = polls_df['approve_adj'] - polls_df['disapprove_adj']

In [116]:
display_df = polls_df[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'net',
                      'total_weights', 'url']]
display_df['sponsors'] = '^Sponsor: ' + display_df['sponsors'] + '^'
display_df['sponsors'] = display_df['sponsors'].fillna('')
display_df = display_df.sort_values(['end_date'], ascending=False)
display_df = display_df.rename({'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 'total_weights':'Weight', 'startdate':'Start Date', 'end_date':'End Date', 'url':'URL'}, axis='columns')
display_df['Sample'] = display_df['sample_size'].astype(int).astype(str) + ' ' + display_df['population']
# display_df['end_date'] = display_df['end_date'].map(lambda x: datetime.datetime.strptime(datetime.datetime.fromtimestamp(x), '%m/%d/%Y').strftime('%Y-%m-%d'))
# display_df['Date'] = pd.to_datetime(display_df['startdate']).astype(str) + ' - ' + display_df['end_date'].astype(str)
display_df['Start Date'] = pd.to_datetime(display_df['Start Date']).astype(str)
display_df['End Date'] = display_df['End Date'].astype(str)
display_df = display_df.reset_index()
# display_df = display_df.drop(['sample_size', 'population'], axis='columns')
display_df['Pollster'] = '<a href="' + display_df['URL'] + '" style="color:black; text-decoration:underline;"><u>' + display_df['Pollster'] + '</u></a>' + display_df['sponsors']
# display_df['Pollster'] = '[' + display_df['Pollster'] + '](' + display_df['URL'] + ')'
display_df = display_df[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net']]
# display_df = display_df.set_index(['Pollster'])
display_df

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net
0,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-05-16,2025-05-19,1710 A,0.069605,43.0,51.0,-8.000000
1,"<a href=""https://insideradvantage.com/trump-jo...",2025-05-17,2025-05-19,1000 LV,0.032260,54.6,44.4,10.200000
2,"<a href=""https://pro.morningconsult.com/tracke...",2025-05-16,2025-05-19,2208 RV,0.013127,48.0,50.0,-2.000000
3,"<a href=""https://www.reuters.com/world/us/trum...",2025-05-16,2025-05-18,1024 A,0.037518,42.0,52.0,-10.000000
4,"<a href=""https://harvardharrispoll.com/key-res...",2025-05-14,2025-05-15,1903 RV,0.042716,47.0,48.0,-1.000000
...,...,...,...,...,...,...,...,...
152,"<a href=""https://echeloninsights.com/in-the-ne...",2025-01-22,2025-01-24,1024 LV,0.000053,51.0,43.0,8.000000
153,"<a href=""https://cdn.atlasintel.org/ed3f5238-e...",2025-01-21,2025-01-23,1882 A,0.000199,50.1,49.7,0.399998
154,"<a href=""https://x.com/ScottWRasmussen/status/...",2025-01-20,2025-01-23,3000 RV,0.000078,57.0,39.0,18.000000
155,"<a href=""https://www.ipsos.com/en-us/reuters-i...",2025-01-20,2025-01-21,1077 A,0.000046,47.0,41.0,6.000000


In [117]:
display_df.to_csv('polls_display_table.csv')

In [118]:
approval_std

,Date,Approve,Disapprove
0,2025-01-23,3.702450,5.577774
1,2025-01-24,3.406119,5.195595
2,2025-01-25,3.406119,5.195595
3,2025-01-26,3.666745,4.786089
4,2025-01-27,3.694118,4.404223
...,...,...,...
113,2025-05-16,3.152507,2.934723
114,2025-05-17,3.152507,2.934723
115,2025-05-18,3.137502,2.878965
116,2025-05-19,3.418836,2.897629


In [119]:
approval_avg['Net'] = approval_avg['Approve'] - approval_avg['Disapprove']
approval_avg

,Date,Approve,Disapprove,Net
0,2025-01-23,51.310566,45.653535,5.657031
1,2025-01-24,51.237797,45.330152,5.907644
2,2025-01-25,51.237797,45.330152,5.907644
3,2025-01-26,50.599984,45.294367,5.305617
4,2025-01-27,49.443088,45.451175,3.991913
...,...,...,...,...
113,2025-05-16,45.222849,50.610528,-5.387679
114,2025-05-17,45.222849,50.610528,-5.387679
115,2025-05-18,45.102544,50.653110,-5.550567
116,2025-05-19,45.308822,50.458753,-5.149931


In [120]:
approval_trendline = pd.merge(left=approval_avg, right=approval_std, left_on='Date', right_on='Date', how='inner')
approval_trendline['approve_CI_upper'] = approval_trendline['Approve_x'] + 1.96*approval_trendline['Approve_y']
approval_trendline['approve_CI_lower'] = approval_trendline['Approve_x'] - 1.96*approval_trendline['Approve_y']
approval_trendline['disapprove_CI_upper'] = approval_trendline['Disapprove_x'] + 1.96*approval_trendline['Disapprove_y']
approval_trendline['disapprove_CI_lower'] = approval_trendline['Disapprove_x'] - 1.96*approval_trendline['Disapprove_y']
approval_trendline = approval_trendline.rename({
    'Approve_x':'approve_mean',
    'Disapprove_x':'disapprove_mean',
}, axis='columns').drop(['Approve_y', 'Disapprove_y', 'Net'], axis='columns')
approval_trendline

,Date,approve_mean,disapprove_mean,approve_CI_upper,approve_CI_lower,disapprove_CI_upper,disapprove_CI_lower
0,2025-01-23,51.310566,45.653535,58.567369,44.053763,56.585972,34.721099
1,2025-01-24,51.237797,45.330152,57.913791,44.561802,55.513519,35.146785
2,2025-01-25,51.237797,45.330152,57.913791,44.561802,55.513519,35.146785
3,2025-01-26,50.599984,45.294367,57.786804,43.413164,54.675102,35.913632
4,2025-01-27,49.443088,45.451175,56.683559,42.202617,54.083453,36.818897
...,...,...,...,...,...,...,...
113,2025-05-16,45.222849,50.610528,51.401762,39.043936,56.362584,44.858471
114,2025-05-17,45.222849,50.610528,51.401762,39.043936,56.362584,44.858471
115,2025-05-18,45.102544,50.653110,51.252047,38.953040,56.295883,45.010338
116,2025-05-19,45.308822,50.458753,52.009741,38.607904,56.138107,44.779400


In [121]:
# Export approval ratings
approval_trendline.to_csv('approval_stats.csv')

In [122]:
polls_df.shape

(157, 20)

In [123]:
line = px.line(approval_avg, x='Date', y=['Approve', 'Disapprove'])
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['approve', 'disapprove'])
line.update_traces(hovertemplate=None)
line.update_layout(hovermode='x unified')
# scatter.update_traces(hovertemplate=None, hoverinfo='skip')
fig_approve_CI = go.Figure([
    go.Scatter(
        name='Approve CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] + 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Approve CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] - 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])
fig_disapprove_CI = go.Figure([
    go.Scatter(
        name='Disapprove CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] + 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Disapprove CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] - 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])

fig = go.Figure(data=line.data + fig_approve_CI.data + fig_disapprove_CI.data)
fig.update_layout(
    title='President Trump Approval Rating',
    xaxis_title='Date',
    yaxis_title='%',
    legend_title='Legend',
    hovermode='x unified'
)
fig.show()

In [141]:
line = px.line(approval_avg, x='Date', y=['Net'])
scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net'])
                     #trendline='lowess', trendline_options=dict(frac=0.2))
fig = go.Figure(data=line.data + scatter.data)
fig.add_hline(y=0, line_dash='dash')
fig.show()

In [125]:
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net', 'net_adj'])
# fig = go.Figure(data=scatter.data)
# fig.add_hline(y=0, line_dash='dash')
# fig.show()

In [147]:
def net_col(df: pd.DataFrame, min_date: datetime.date):
#     df_copy = df.set_index(['Date'])
#     return df_copy['approve_mean'] - df_copy['disapprove_mean']
    date_range = pd.date_range(start=min_date, end=datetime.datetime.today(), freq='d', inclusive='both')
    data = df.copy()
    data['net_adj'] = data['approve_adj'] - data['disapprove_adj']
    net_averages = []
    net_stdevs = []
    issue_cols = {'Date': date_range, 'net':[], 'net_std':[]}
    for date in date_range:
    #         try:
            pipelined_df = avgs_pipeline(data, date)
    #         except TypeError:
    #             print('TypeError')
    #             print(date)
            net_avg = np.sum(pipelined_df['net_adj'] * pipelined_df['total_weights'])
            net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net_adj'] - net_avg)**2))

            net_averages.append(net_avg)
            net_stdevs.append(net_std)

    issue_cols['net'].extend(net_averages)
    issue_cols[f'net_std'].extend(net_stdevs)
        
    net_averages, net_stdevs = [], []
        
    return pd.DataFrame(issue_cols)

In [149]:
net_approval = net_col(approval, datetime.date(2025, 1, 23))
net_approval

,Date,net,net_std
0,2025-01-23,5.657031,8.630764
1,2025-01-24,5.907644,7.966501
2,2025-01-25,5.907644,7.966501
3,2025-01-26,5.305617,7.611568
4,2025-01-27,3.991913,7.074258
...,...,...,...
113,2025-05-16,-5.387679,5.407211
114,2025-05-17,-5.387679,5.407211
115,2025-05-18,-5.550567,5.347689
116,2025-05-19,-5.149931,5.723985


In [151]:
net_approval['net_upper_CI'] = net_approval['net'] + 1.96*net_approval['net_std']
net_approval['net_lower_CI'] = net_approval['net'] - 1.96*net_approval['net_std']
net_approval

,Date,net,net_std,net_upper_CI,net_lower_CI
0,2025-01-23,5.657031,8.630764,22.573328,-11.259267
1,2025-01-24,5.907644,7.966501,21.521987,-9.706698
2,2025-01-25,5.907644,7.966501,21.521987,-9.706698
3,2025-01-26,5.305617,7.611568,20.224291,-9.613057
4,2025-01-27,3.991913,7.074258,17.857460,-9.873633
...,...,...,...,...,...
113,2025-05-16,-5.387679,5.407211,5.210455,-15.985813
114,2025-05-17,-5.387679,5.407211,5.210455,-15.985813
115,2025-05-18,-5.550567,5.347689,4.930904,-16.032038
116,2025-05-19,-5.149931,5.723985,6.069079,-16.368941


In [152]:
net_approval.to_csv('net_approval.csv')